# Libraries

In [43]:
from contextlib import contextmanager
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None


# Helper Functions

In [44]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


# application_train

In [45]:
def application_train():

    df = pd.read_csv("input/application_train.csv")
    test_df = pd.read_csv("input/application_test.csv")

    # df = df.append(test_df).reset_index()
    df = pd.concat([df, test_df], ignore_index=True).reset_index()
    df = df[df["CODE_GENDER"] != "XNA"]

    lbe = LabelEncoder()

    for col in ["CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY"]:
        df[col] = lbe.fit_transform(df[col])

    df = pd.get_dummies(df, dummy_na=True)

    df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)
    df["NEW_DAYS_EMPLOYED_PERC"] = df["DAYS_EMPLOYED"] / df["DAYS_BIRTH"]
    df["NEW_INCOME_CREDIT_PERC"] = df["AMT_INCOME_TOTAL"] / df["AMT_CREDIT"]
    df["NEW_INCOME_PER_PERSON"] = df["AMT_INCOME_TOTAL"] / df["CNT_FAM_MEMBERS"]
    df["NEW_ANNUITY_INCOME_PERC"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]
    df["NEW_PAYMENT_RATE"] = df["AMT_ANNUITY"] / df["AMT_CREDIT"]

    df.drop("index", axis=1, inplace=True)

    df.columns = pd.Index(["APP_" + col for col in df.columns.tolist()])

    df.rename(columns={"APP_SK_ID_CURR": "SK_ID_CURR"}, inplace=True)

    df.rename(columns={"APP_TARGET": "TARGET"}, inplace=True)

    return df


# bureau & bureau_balance

In [46]:
def bureau_bb():

    # reading the table bureau_balance

    bb = pd.read_csv("input/bureau_balance.csv")
    bb = pd.get_dummies(bb, dummy_na=True)

    agg_list = {
        "MONTHS_BALANCE": "count",
        "STATUS_0": ["sum", "mean"],
        "STATUS_1": ["sum"],
        "STATUS_2": ["sum"],
        "STATUS_3": ["sum"],
        "STATUS_4": ["sum"],
        "STATUS_5": ["sum"],
        "STATUS_C": ["sum", "mean"],
        "STATUS_X": ["sum", "mean"],
    }

    bb_agg = bb.groupby("SK_ID_BUREAU").agg(agg_list)

    # Renaming variables
    bb_agg.columns = pd.Index(
        [col[0] + "_" + col[1].upper() for col in bb_agg.columns.tolist()]
    )

    # Creating a new variable for status_sum
    bb_agg["NEW_STATUS_SCORE"] = (
        bb_agg["STATUS_1_SUM"] + bb_agg["STATUS_2_SUM"]
        ^ 2 + bb_agg["STATUS_3_SUM"]
        ^ 3 + bb_agg["STATUS_4_SUM"]
        ^ 4 + bb_agg["STATUS_5_SUM"]
        ^ 5
    )

    bb_agg.drop(
        [
            "STATUS_1_SUM",
            "STATUS_2_SUM",
            "STATUS_3_SUM",
            "STATUS_4_SUM",
            "STATUS_5_SUM",
        ],
        axis=1,
        inplace=True,
    )

    bureau = pd.read_csv("input/bureau.csv")
    bureau_and_bb = bureau.join(bb_agg, how="left", on="SK_ID_BUREAU")

    # BUREAU BALANCE AND BUREAU COMMON TABLE

    # CREDIT_TYPE reduce the class number of the variable to 3
    bureau_and_bb["CREDIT_TYPE"] = bureau_and_bb["CREDIT_TYPE"].replace(
        [
            "Car loan",
            "Mortgage",
            "Microloan",
            "Loan for business development",
            "Another type of loan",
            "Unknown type of loan",
            "Loan for working capital replenishment",
            "Loan for purchase of shares (margin lending)",
            "Cash loan (non-earmarked)",
            "Real estate loan",
            "Loan for the purchase of equipment",
            "Interbank credit",
            "Mobile operator loan",
        ],
        "Rare",
    )

    # CREDIT_ACTIVE reducing the class number of the variable to 2 (Would it be more appropriate to include Left in Closed???)
    bureau_and_bb["CREDIT_ACTIVE"] = bureau_and_bb["CREDIT_ACTIVE"].replace(
        ["Bad debt", "Sold"], "Active"
    )

    # bureau_bb applying One Hot Encoding to categorical variables in the table
    bureau_and_bb = pd.get_dummies(
        bureau_and_bb, columns=["CREDIT_TYPE", "CREDIT_ACTIVE"]
    )

    # CREDIT_CURRENCY 99% of the variable is currency 1, so we deleted it because we thought it would not be distinctive.
    bureau_and_bb.drop(["SK_ID_BUREAU", "CREDIT_CURRENCY"], inplace=True, axis=1)

    # NEW FEATURES

    # new variable showing how many months of average loan you get
    bureau_and_bb["NEW_MONTHS_CREDIT"] = round(
        (bureau_and_bb.DAYS_CREDIT_ENDDATE - bureau_and_bb.DAYS_CREDIT) / 30
    )

    agg_list = {
        "SK_ID_CURR": ["count"],
        "DAYS_CREDIT": ["min", "max"],
        "CREDIT_DAY_OVERDUE": ["sum", "mean", "max"],
        "DAYS_CREDIT_ENDDATE": ["max", "min"],
        "DAYS_ENDDATE_FACT": ["max", "min"],
        "AMT_CREDIT_MAX_OVERDUE": ["mean", "max", "min"],
        "CNT_CREDIT_PROLONG": ["sum", "mean", "max", "min"],
        "AMT_CREDIT_SUM": ["mean", "max", "min"],
        "AMT_CREDIT_SUM_DEBT": ["sum", "mean", "max"],
        "AMT_CREDIT_SUM_LIMIT": ["sum", "mean", "max"],
        "AMT_CREDIT_SUM_OVERDUE": ["sum", "mean", "max"],
        "DAYS_CREDIT_UPDATE": ["max", "min"],
        "AMT_ANNUITY": ["sum", "mean"],
        "MONTHS_BALANCE_COUNT": ["sum"],
        "STATUS_0_SUM": ["sum"],
        "STATUS_0_MEAN": ["mean"],
        "STATUS_C_SUM": ["sum"],
        "STATUS_C_MEAN": ["mean"],
        "CREDIT_ACTIVE_Active": ["sum", "mean"],
        "CREDIT_ACTIVE_Closed": ["sum", "mean"],
        "CREDIT_TYPE_Rare": ["sum", "mean"],
        "CREDIT_TYPE_Consumer credit": ["sum", "mean"],
        "CREDIT_TYPE_Credit card": ["sum", "mean"],
        "NEW_MONTHS_CREDIT": ["count", "sum", "mean", "max", "min"],
    }

    # Applying aggregation operations to bureau _bb_agg table
    bureau_and_bb_agg = bureau_and_bb.groupby("SK_ID_CURR").agg(agg_list).reset_index()

    # Renaming variables
    bureau_and_bb_agg.columns = pd.Index(
        [
            "BB_" + col[0] + "_" + col[1].upper()
            for col in bureau_and_bb_agg.columns.tolist()
        ]
    )

    # A new variable that shows the difference between the highest and lowest credit received by a person.
    bureau_and_bb_agg["BB_NEW_AMT_CREDIT_SUM_RANGE"] = (
        bureau_and_bb_agg["BB_AMT_CREDIT_SUM_MAX"]
        - bureau_and_bb_agg["BB_AMT_CREDIT_SUM_MIN"]
    )

    # The new variable that expresses the average number of months to take out a loan.
    bureau_and_bb_agg["BB_NEW_DAYS_CREDIT_RANGE"] = round(
        (
            bureau_and_bb_agg["BB_DAYS_CREDIT_MAX"]
            - bureau_and_bb_agg["BB_DAYS_CREDIT_MIN"]
        )
        / (30 * bureau_and_bb_agg["BB_SK_ID_CURR_COUNT"])
    )

    # Bureau: Active credits - using only numerical aggregations
    agg_list = {
        "DAYS_CREDIT": ["min", "max", "mean", "var"],
        "DAYS_CREDIT_ENDDATE": ["min", "max", "mean"],
        "DAYS_CREDIT_UPDATE": ["mean"],
        "CREDIT_DAY_OVERDUE": ["max", "mean"],
        "AMT_CREDIT_MAX_OVERDUE": ["mean"],
        "AMT_CREDIT_SUM": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_DEBT": ["max", "mean", "sum"],
        "AMT_CREDIT_SUM_OVERDUE": ["mean"],
        "AMT_CREDIT_SUM_LIMIT": ["mean", "sum"],
        "AMT_ANNUITY": ["max", "mean"],
        "CNT_CREDIT_PROLONG": ["sum"],
    }

    active = bureau_and_bb[bureau_and_bb["CREDIT_ACTIVE_Active"] == 1]
    active_agg = active.groupby("SK_ID_CURR").agg(agg_list)
    active_agg.columns = pd.Index(
        [
            "BB_NEW_ACTIVE_" + e[0] + "_" + e[1].upper()
            for e in active_agg.columns.tolist()
        ]
    )
    bureau_and_bb_agg.rename(columns={"BB_SK_ID_CURR_": "SK_ID_CURR"}, inplace=True)
    bureau_and_bb_agg = bureau_and_bb_agg.join(active_agg, how="left", on="SK_ID_CURR")

    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau_and_bb[bureau_and_bb["CREDIT_ACTIVE_Closed"] == 1]
    closed_agg = closed.groupby("SK_ID_CURR").agg(agg_list)
    closed_agg.columns = pd.Index(
        [
            "BB_NEW_CLOSED_" + e[0] + "_" + e[1].upper()
            for e in closed_agg.columns.tolist()
        ]
    )
    bureau_and_bb_agg = bureau_and_bb_agg.join(closed_agg, how="left", on="SK_ID_CURR")

    return bureau_and_bb_agg


# installments_payments

In [47]:
def installments_payments():

    # Read the installments_payments.csv
    ins = pd.read_csv("input/installments_payments.csv")

    ins["NEW_DAYS_PAID_EARLIER"] = ins["DAYS_INSTALMENT"] - ins["DAYS_ENTRY_PAYMENT"]

    # Whether each installment payment is late or not 1: late paid 0: represents early payment
    ins["NEW_NUM_PAID_LATER"] = ins["NEW_DAYS_PAID_EARLIER"].map(
        lambda x: 1 if x < 0 else 0
    )

    # Agrregation and variable deduplication
    agg_list = {
        "NUM_INSTALMENT_VERSION": ["nunique"],
        "NUM_INSTALMENT_NUMBER": "max",
        "DAYS_INSTALMENT": ["min", "max"],
        "DAYS_ENTRY_PAYMENT": ["min", "max"],
        "AMT_INSTALMENT": ["min", "max", "sum", "mean"],
        "AMT_PAYMENT": ["min", "max", "sum", "mean"],
        "NEW_DAYS_PAID_EARLIER": "mean",
        "NEW_NUM_PAID_LATER": "sum",
    }

    ins_agg = ins.groupby("SK_ID_PREV").agg(agg_list)

    # Multi index problem solution
    ins_agg.columns = pd.Index(
        ["INS_" + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()]
    )

    # drop variables
    ins_agg.drop(
        [
            "INS_DAYS_INSTALMENT_MIN",
            "INS_DAYS_INSTALMENT_MAX",
            "INS_DAYS_ENTRY_PAYMENT_MIN",
            "INS_DAYS_ENTRY_PAYMENT_MAX",
        ],
        axis=1,
        inplace=True,
    )

    # Loan payment percentage and total remaining debt
    ins_agg["INS_NEW_PAYMENT_PERC"] = (
        ins_agg["INS_AMT_PAYMENT_SUM"] / ins_agg["INS_AMT_INSTALMENT_SUM"]
    )
    ins_agg["INS_NEW_PAYMENT_DIFF"] = (
        ins_agg["INS_AMT_INSTALMENT_SUM"] - ins_agg["INS_AMT_PAYMENT_SUM"]
    )

    agg_list_previous_application = {}

    for col in ins_agg.columns:
        agg_list_previous_application[col] = ["mean", "min", "max", "sum"]

    ins_agg.reset_index(inplace=True)

    return agg_list_previous_application, ins_agg


# pos_cash_balance

In [48]:
def pos_cash_balance(agg_list_previous_application):

    pos = pd.read_csv("input/POS_CASH_balance.csv")
    # Converting Our Categorical Variable to a Dummy Variable
    pos = pd.get_dummies(pos, columns=["NAME_CONTRACT_STATUS"], dummy_na=True)
    # Aggregation Process - Deduplication
    agg_list = {
        "MONTHS_BALANCE": ["min", "max"],
        "CNT_INSTALMENT": ["min", "max"],
        "CNT_INSTALMENT_FUTURE": ["min", "max"],
        "SK_DPD": ["max", "mean"],
        "SK_DPD_DEF": ["max", "mean"],
        "NAME_CONTRACT_STATUS_Active": "sum",
        "NAME_CONTRACT_STATUS_Amortized debt": "sum",
        "NAME_CONTRACT_STATUS_Approved": "sum",
        "NAME_CONTRACT_STATUS_Canceled": "sum",
        "NAME_CONTRACT_STATUS_Completed": "sum",
        "NAME_CONTRACT_STATUS_Demand": "sum",
        "NAME_CONTRACT_STATUS_Returned to the store": "sum",
        "NAME_CONTRACT_STATUS_Signed": "sum",
        "NAME_CONTRACT_STATUS_XNA": "sum",
        "NAME_CONTRACT_STATUS_nan": "sum",
    }

    pos_agg = pos.groupby("SK_ID_PREV").agg(agg_list)

    # Converting multilayer index to one dimensional index
    pos_agg.columns = pd.Index(
        ["POS_" + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()]
    )

    # SK_DPD is 0 in how many credits (SK_DPD MAX gives the status of 0)
    # SK_DPD_DEF (SK_DPD_DEF_MAX returns zero)
    # Status of NAME_CONTRACT_STATUS_Completed_SUM==0 when CNT_INSTALMENT_FUTURE_MIN==0

    pos_agg["POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME"] = (
        pos_agg["POS_CNT_INSTALMENT_FUTURE_MIN"] == 0
    ) & (pos_agg["POS_NAME_CONTRACT_STATUS_Completed_SUM"] == 0)

    # 1: not closed on loan time 0: closed on loan time

    pos_agg["POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME"] = pos_agg[
        "POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME"
    ].astype(int)

    pos_agg.drop(
        [
            "POS_NAME_CONTRACT_STATUS_Approved_SUM",
            "POS_NAME_CONTRACT_STATUS_Amortized debt_SUM",
            "POS_NAME_CONTRACT_STATUS_Canceled_SUM",
            "POS_NAME_CONTRACT_STATUS_Returned to the store_SUM",
            "POS_NAME_CONTRACT_STATUS_Signed_SUM",
            "POS_NAME_CONTRACT_STATUS_XNA_SUM",
            "POS_NAME_CONTRACT_STATUS_nan_SUM",
        ],
        axis=1,
        inplace=True,
    )

    for col in pos_agg.columns:
        agg_list_previous_application[col] = ["mean", "min", "max", "sum"]

    pos_agg.reset_index(inplace=True)

    return agg_list_previous_application, pos_agg


# credit_card_balance

In [49]:
def credit_card_balance():

    CCB = pd.read_csv("input/credit_card_balance.csv")

    CCB = pd.get_dummies(CCB, columns=["NAME_CONTRACT_STATUS"])  # artik tumu sayisal

    dropthis = [
        "NAME_CONTRACT_STATUS_Approved",
        "NAME_CONTRACT_STATUS_Demand",
        "NAME_CONTRACT_STATUS_Refused",
        "NAME_CONTRACT_STATUS_Sent proposal",
        "NAME_CONTRACT_STATUS_Signed",
    ]

    CCB = CCB.drop(dropthis, axis=1)

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["SK_ID_PREV"]
        .nunique()
        .reset_index()
        .rename(index=str, columns={"SK_ID_PREV": "NUMBER_OF_LOANS_PER_CUSTOMER"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    grp = (
        CCB.groupby(by=["SK_ID_CURR", "SK_ID_PREV"])["CNT_INSTALMENT_MATURE_CUM"]
        .max()
        .reset_index()
        .rename(
            index=str, columns={"CNT_INSTALMENT_MATURE_CUM": "NUMBER_OF_INSTALMENTS"}
        )
    )
    grp1 = (
        grp.groupby(by=["SK_ID_CURR"])["NUMBER_OF_INSTALMENTS"]
        .sum()
        .reset_index()
        .rename(
            index=str,
            columns={"NUMBER_OF_INSTALMENTS": "TOTAL_INSTALMENTS_OF_ALL_LOANS"},
        )
    )
    CCB = CCB.merge(grp1, on=["SK_ID_CURR"], how="left")

    CCB["INSTALLMENTS_PER_LOAN"] = (
        CCB["TOTAL_INSTALMENTS_OF_ALL_LOANS"] / CCB["NUMBER_OF_LOANS_PER_CUSTOMER"]
    ).astype("uint32")

    # Function to calculate number of times Days Past Due occurred
    def geciken_gun_hesapla(DPD):

        # An expected series with DPD: Values of SK DPD variable for historical credit in each prev_app
        # DPD is a series of values of SK_DPD for each of the groupby combination
        # We convert it to a list to get the number of SK_DPD values NOT EQUALS ZERO
        x = DPD.tolist()
        c = 0
        for i, j in enumerate(x):
            if j != 0:
                c += 1
        return c

    grp = (
        CCB.groupby(by=["SK_ID_CURR", "SK_ID_PREV"])
        .apply(lambda x: geciken_gun_hesapla(x.SK_DPD))
        .reset_index()
        .rename(index=str, columns={0: "NUMBER_OF_DPD"})
    )
    grp1 = (
        grp.groupby(by=["SK_ID_CURR"])["NUMBER_OF_DPD"]
        .mean()
        .reset_index()
        .rename(index=str, columns={"NUMBER_OF_DPD": "DPD_COUNT"})
    )

    CCB = CCB.merge(grp1, on=["SK_ID_CURR"], how="left")

    def f(min_pay, total_pay):

        M = min_pay.tolist()
        T = total_pay.tolist()
        P = len(M)  # P: number of installments
        c = 0
        # Find the count of transactions when Payment made is less than Minimum Payment
        for i in range(len(M)):
            if T[i] < M[i]:
                c += 1
        return (100 * c) / P

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])
        .apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT))
        .reset_index()
        .rename(index=str, columns={0: "PERCENTAGE_MIN_MISSED_PAYMENTS"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["AMT_DRAWINGS_ATM_CURRENT"]
        .sum()
        .reset_index()
        .rename(index=str, columns={"AMT_DRAWINGS_ATM_CURRENT": "DRAWINGS_ATM"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["AMT_DRAWINGS_CURRENT"]
        .sum()
        .reset_index()
        .rename(index=str, columns={"AMT_DRAWINGS_CURRENT": "DRAWINGS_TOTAL"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    CCB["CASH_CARD_RATIO1"] = (
        CCB["DRAWINGS_ATM"] / CCB["DRAWINGS_TOTAL"]
    ) * 100  # Cash withdrawn from ATM / total withdrawal
    del CCB["DRAWINGS_ATM"]
    del CCB["DRAWINGS_TOTAL"]

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["CASH_CARD_RATIO1"]
        .mean()
        .reset_index()
        .rename(index=str, columns={"CASH_CARD_RATIO1": "CASH_CARD_RATIO"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["AMT_DRAWINGS_CURRENT"]
        .sum()
        .reset_index()
        .rename(index=str, columns={"AMT_DRAWINGS_CURRENT": "TOTAL_DRAWINGS"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["CNT_DRAWINGS_CURRENT"]
        .sum()
        .reset_index()
        .rename(index=str, columns={"CNT_DRAWINGS_CURRENT": "NUMBER_OF_DRAWINGS"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    CCB["DRAWINGS_RATIO1"] = (
        CCB["TOTAL_DRAWINGS"] / CCB["NUMBER_OF_DRAWINGS"]
    ) * 100  # not a percentile, it's enlarged
    del CCB["TOTAL_DRAWINGS"]
    del CCB["NUMBER_OF_DRAWINGS"]

    grp = (
        CCB.groupby(by=["SK_ID_CURR"])["DRAWINGS_RATIO1"]
        .mean()
        .reset_index()
        .rename(index=str, columns={"DRAWINGS_RATIO1": "DRAWINGS_RATIO"})
    )
    CCB = CCB.merge(grp, on=["SK_ID_CURR"], how="left")

    del CCB["DRAWINGS_RATIO1"]

    CCB["CC_COUNT"] = CCB.groupby("SK_ID_CURR").size()

    CCB_agg = CCB.groupby("SK_ID_CURR").agg(
        {
            "MONTHS_BALANCE": ["sum", "mean"],
            "AMT_BALANCE": ["sum", "mean", "min", "max"],
            "AMT_CREDIT_LIMIT_ACTUAL": ["sum", "mean"],
            "AMT_DRAWINGS_ATM_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_DRAWINGS_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_DRAWINGS_OTHER_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_DRAWINGS_POS_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_INST_MIN_REGULARITY": ["sum", "mean", "min", "max"],
            "AMT_PAYMENT_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_PAYMENT_TOTAL_CURRENT": ["sum", "mean", "min", "max"],
            "AMT_RECEIVABLE_PRINCIPAL": ["sum", "mean", "min", "max"],
            "AMT_RECIVABLE": ["sum", "mean", "min", "max"],
            "AMT_TOTAL_RECEIVABLE": ["sum", "mean", "min", "max"],
            "CNT_DRAWINGS_ATM_CURRENT": ["sum", "mean"],
            "CNT_DRAWINGS_CURRENT": ["sum", "mean", "max"],
            "CNT_DRAWINGS_OTHER_CURRENT": ["mean", "max"],
            "CNT_DRAWINGS_POS_CURRENT": ["sum", "mean", "max"],
            "CNT_INSTALMENT_MATURE_CUM": ["sum", "mean", "max", "min"],
            "SK_DPD": ["sum", "mean", "max"],
            "SK_DPD_DEF": ["sum", "mean", "max"],
            "NAME_CONTRACT_STATUS_Active": ["sum", "mean", "min", "max"],
            "INSTALLMENTS_PER_LOAN": ["sum", "mean", "min", "max"],
            "NUMBER_OF_LOANS_PER_CUSTOMER": ["mean"],
            "DPD_COUNT": ["mean"],
            "PERCENTAGE_MIN_MISSED_PAYMENTS": ["mean"],
            "CASH_CARD_RATIO": ["mean"],
            "DRAWINGS_RATIO": ["mean"],
        }
    )

    CCB_agg.columns = pd.Index(
        ["CCB_" + e[0] + "_" + e[1].upper() for e in CCB_agg.columns.tolist()]
    )

    CCB_agg.reset_index(inplace=True)

    return CCB_agg


# previous_application

In [50]:
def previous_application(agg_list_previous_application):

    df_prev = pd.read_csv("input/previous_application.csv")

    # "WEEKDAY_APPR_PROCESS_START"  dividing the variable into two categories as WEEK_DAY and WEEKEND

    df_prev["WEEKDAY_APPR_PROCESS_START"] = df_prev[
        "WEEKDAY_APPR_PROCESS_START"
    ].replace(["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY"], "WEEK_DAY")
    df_prev["WEEKDAY_APPR_PROCESS_START"] = df_prev[
        "WEEKDAY_APPR_PROCESS_START"
    ].replace(["SATURDAY", "SUNDAY"], "WEEKEND")

    # "HOUR_APPR_PROCESS_START"  dividing the variable into two categories, working_hours and off_hours
    a = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    df_prev["HOUR_APPR_PROCESS_START"] = df_prev["HOUR_APPR_PROCESS_START"].replace(
        a, "working_hours"
    )

    b = [18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7]
    df_prev["HOUR_APPR_PROCESS_START"] = df_prev["HOUR_APPR_PROCESS_START"].replace(
        b, "off_hours"
    )

    # Those with DAYS_DECISION values less than 1 year were assigned a value of 1 and a value greater than 0.
    df_prev["DAYS_DECISION"] = [
        1 if abs(i / (12 * 30)) <= 1 else 0 for i in df_prev.DAYS_DECISION
    ]

    # Splitting the "NAME_TYPE_SUITE" variable into two categories, alone and not_alone

    df_prev["NAME_TYPE_SUITE"] = df_prev["NAME_TYPE_SUITE"].replace(
        "Unaccompanied", "alone"
    )

    b = [
        "Family",
        "Spouse, partner",
        "Children",
        "Other_B",
        "Other_A",
        "Group of people",
    ]
    df_prev["NAME_TYPE_SUITE"] = df_prev["NAME_TYPE_SUITE"].replace(b, "not_alone")

    # These values in the variable "NAME_GOODS_CATEGORY" will be categorized as others
    a = [
        "Auto Accessories",
        "Jewelry",
        "Homewares",
        "Medical Supplies",
        "Vehicles",
        "Sport and Leisure",
        "Gardening",
        "Other",
        "Office Appliances",
        "Tourism",
        "Medicine",
        "Direct Sales",
        "Fitness",
        "Additional Service",
        "Education",
        "Weapon",
        "Insurance",
        "House Construction",
        "Animals",
    ]
    df_prev["NAME_GOODS_CATEGORY"] = df_prev["NAME_GOODS_CATEGORY"].replace(a, "others")

    # These values in the "NAME_SELLER_INDUSTRY" variable will be categorized as others
    a = ["Auto technology", "Jewelry", "MLM partners", "Tourism"]
    df_prev["NAME_SELLER_INDUSTRY"] = df_prev["NAME_SELLER_INDUSTRY"].replace(
        a, "others"
    )
    # Derives the variable containing the ratio of the loan requested to the loan given
    df_prev["LOAN_RATE"] = df_prev.AMT_APPLICATION / df_prev.AMT_CREDIT

    # YENI DEGISKENLER

    # Derives the variable containing the ratio of the loan requested to the loan given
    df_prev["NEW_LOAN_RATE"] = df_prev.AMT_APPLICATION / df_prev.AMT_CREDIT

    # The churn_prev variable is derived, which indicates whether the payment has been delayed.
    # 1= delayed, 0 = not delayed, NaN = null
    k = df_prev.DAYS_LAST_DUE_1ST_VERSION - df_prev.DAYS_LAST_DUE
    df_prev["NEW_CHURN_PREV"] = [1 if i >= 0 else (0 if i < 0 else "NaN") for i in k]

    # NEW_INSURANCE variable is defined to be used instead of NFLAG_INSURED_ON_APPROVAL variable.
    df_prev[(df_prev["AMT_CREDIT"] == 0) | (df_prev["AMT_GOODS_PRICE"] == 0)][
        "NEW_INSURANCE"
    ] = np.nan
    df_prev["sigorta_miktari"] = df_prev["AMT_CREDIT"] - df_prev["AMT_GOODS_PRICE"]
    df_prev["NEW_INSURANCE"] = df_prev["sigorta_miktari"].apply(
        lambda x: 1 if x > 0 else (0 if x <= 0 else np.nan)
    )
    df_prev.drop("sigorta_miktari", axis=1, inplace=True)

    # Creates the INTEREST_RATE variable.
    # df_prev['INTEREST_RATE'] = (df_prev.AMT_ANNUITY*df_prev.CNT_PAYMENT/df_prev.AMT_CREDIT)**(12/df_prev.CNT_PAYMENT)-1
    # df_prev[df_prev['INTEREST_RATE']==-1]=np.nan

    drop_list = [
        "AMT_DOWN_PAYMENT",
        "SELLERPLACE_AREA",
        "CNT_PAYMENT",
        "PRODUCT_COMBINATION",
        "DAYS_FIRST_DRAWING",
        "DAYS_FIRST_DUE",
        "DAYS_LAST_DUE_1ST_VERSION",
        "DAYS_LAST_DUE",
        "DAYS_TERMINATION",
        "NFLAG_INSURED_ON_APPROVAL",
    ]
    df_prev.drop(drop_list, axis=1, inplace=True)

    # Holds the names of the categorical variables in the previous table.
    category_columns = []
    for i in df_prev.columns:
        if df_prev[i].dtypes == "O":
            category_columns.append(i)

    df_prev = pd.get_dummies(df_prev, columns=category_columns)

    prev_agg_list = {
        "SK_ID_CURR": ["count"],
        "AMT_ANNUITY": ["max"],
        "AMT_APPLICATION": ["min", "mean", "max"],
        "AMT_CREDIT": ["max"],
        "AMT_GOODS_PRICE": ["sum", "mean"],
        "NFLAG_LAST_APPL_IN_DAY": ["sum", "mean"],
        "RATE_DOWN_PAYMENT": ["sum", "mean"],
        "RATE_INTEREST_PRIMARY": ["sum", "mean"],
        "RATE_INTEREST_PRIVILEGED": ["sum", "mean"],
        "DAYS_DECISION": ["sum"],
        "NEW_LOAN_RATE": ["sum", "mean", "min", "max"],
        "NEW_INSURANCE": ["sum", "mean"],
        # "INTEREST_RATE":["sum", "mean", "min", "max"],
        "NAME_CONTRACT_TYPE_Cash loans": ["sum", "mean"],
        "NAME_CONTRACT_TYPE_Consumer loans": ["sum", "mean"],
        "NAME_CONTRACT_TYPE_Revolving loans": ["sum", "mean"],
        "NAME_CONTRACT_TYPE_XNA": ["sum", "mean"],
        "WEEKDAY_APPR_PROCESS_START_WEEKEND": ["sum", "mean"],
        "WEEKDAY_APPR_PROCESS_START_WEEK_DAY": ["sum", "mean"],
        "HOUR_APPR_PROCESS_START_off_hours": ["sum", "mean"],
        "HOUR_APPR_PROCESS_START_working_hours": ["sum", "mean"],
        "FLAG_LAST_APPL_PER_CONTRACT_N": ["sum", "mean"],
        "FLAG_LAST_APPL_PER_CONTRACT_Y": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Building a house or an annex": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Business development": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Buying a garage": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Buying a home": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Buying a new car": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Buying a used car": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Car repairs": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Education": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Everyday expenses": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Furniture": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Gasification / water supply": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Hobby": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Journey": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Medicine": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Money for a third person": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Other": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Payments on other loans": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Refusal to name the goal": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Repairs": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Urgent needs": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_XAP": ["sum", "mean"],
        "NAME_CASH_LOAN_PURPOSE_XNA": ["sum", "mean"],
        "NAME_CONTRACT_STATUS_Approved": ["sum", "mean"],
        "NAME_CONTRACT_STATUS_Canceled": ["sum", "mean"],
        "NAME_CONTRACT_STATUS_Refused": ["sum", "mean"],
        "NAME_CONTRACT_STATUS_Unused offer": ["sum", "mean"],
        "NAME_PAYMENT_TYPE_Cash through the bank": ["sum", "mean"],
        "NAME_PAYMENT_TYPE_Cashless from the account of the employer": ["sum", "mean"],
        "NAME_PAYMENT_TYPE_Non-cash from your account": ["sum", "mean"],
        "NAME_PAYMENT_TYPE_XNA": ["sum", "mean"],
        "CODE_REJECT_REASON_CLIENT": ["sum", "mean"],
        "CODE_REJECT_REASON_HC": ["sum", "mean"],
        "CODE_REJECT_REASON_LIMIT": ["sum", "mean"],
        "CODE_REJECT_REASON_SCO": ["sum", "mean"],
        "CODE_REJECT_REASON_SCOFR": ["sum", "mean"],
        "CODE_REJECT_REASON_SYSTEM": ["sum", "mean"],
        "CODE_REJECT_REASON_VERIF": ["sum", "mean"],
        "CODE_REJECT_REASON_XAP": ["sum", "mean"],
        "CODE_REJECT_REASON_XNA": ["sum", "mean"],
        "NAME_TYPE_SUITE_alone": ["sum", "mean"],
        "NAME_TYPE_SUITE_not_alone": ["sum", "mean"],
        "NAME_CLIENT_TYPE_New": ["sum", "mean"],
        "NAME_CLIENT_TYPE_Refreshed": ["sum", "mean"],
        "NAME_CLIENT_TYPE_Repeater": ["sum", "mean"],
        "NAME_CLIENT_TYPE_XNA": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Audio/Video": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Clothing and Accessories": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Computers": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Construction Materials": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Consumer Electronics": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Furniture": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Mobile": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_Photo / Cinema Equipment": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_XNA": ["sum", "mean"],
        "NAME_GOODS_CATEGORY_others": ["sum", "mean"],
        "NAME_PORTFOLIO_Cards": ["sum", "mean"],
        "NAME_PORTFOLIO_Cars": ["sum", "mean"],
        "NAME_PORTFOLIO_Cash": ["sum", "mean"],
        "NAME_PORTFOLIO_POS": ["sum", "mean"],
        "NAME_PORTFOLIO_XNA": ["sum", "mean"],
        "NAME_PRODUCT_TYPE_XNA": ["sum", "mean"],
        "NAME_PRODUCT_TYPE_walk-in": ["sum", "mean"],
        "NAME_PRODUCT_TYPE_x-sell": ["sum", "mean"],
        "CHANNEL_TYPE_AP+ (Cash loan)": ["sum", "mean"],
        "CHANNEL_TYPE_Car dealer": ["sum", "mean"],
        "CHANNEL_TYPE_Channel of corporate sales": ["sum", "mean"],
        "CHANNEL_TYPE_Contact center": ["sum", "mean"],
        "CHANNEL_TYPE_Country-wide": ["sum", "mean"],
        "CHANNEL_TYPE_Credit and cash offices": ["sum", "mean"],
        "CHANNEL_TYPE_Regional / Local": ["sum", "mean"],
        "CHANNEL_TYPE_Stone": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Clothing": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Connectivity": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Construction": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Consumer electronics": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Furniture": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_Industry": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_XNA": ["sum", "mean"],
        "NAME_SELLER_INDUSTRY_others": ["sum", "mean"],
        "NAME_YIELD_GROUP_XNA": ["sum", "mean"],
        "NAME_YIELD_GROUP_high": ["sum", "mean"],
        "NAME_YIELD_GROUP_low_action": ["sum", "mean"],
        "NAME_YIELD_GROUP_low_normal": ["sum", "mean"],
        "NAME_YIELD_GROUP_middle": ["sum", "mean"],
        "NEW_CHURN_PREV_0": ["sum", "mean"],
        "NEW_CHURN_PREV_1": ["sum", "mean"],
        "NEW_CHURN_PREV_NaN": ["sum", "mean"],
    }

    prev_agg_list.update(agg_list_previous_application)

    return prev_agg_list, df_prev


# Combine

In [51]:
def pre_processing_and_combine():

    with timer("Process application train"):
        df = application_train()
        print("application train & test shape:", df.shape)

    with timer("Bureau and Bureau Balance"):
        bureau_and_bb_agg = bureau_bb()
        print("Bureau and Bureau Balance:", bureau_and_bb_agg.shape)

    with timer("Installment Payments"):
        agg_list_previous_application, ins_agg = installments_payments()
        print("Installment Payments:", ins_agg.shape)

    with timer("Pos Cash Balance"):
        agg_list_previous_application, pos_agg = pos_cash_balance(
            agg_list_previous_application
        )
        print("Pos Cash Balance:", pos_agg.shape)

    with timer("Credit Card Balance"):
        CCB_agg = credit_card_balance()
        print("Credit Card Balance:", CCB_agg.shape)

    with timer("previous_application"):
        prev_agg_list, df_prev = previous_application(agg_list_previous_application)
        print("previous_application:", df_prev.shape)

    with timer("All tables are combining"):
        df_prev_ins = df_prev.merge(ins_agg, how="left", on="SK_ID_PREV")
        df_prev_ins_pos = df_prev_ins.merge(pos_agg, how="left", on="SK_ID_PREV")
        df_prev_ins_pos_agg = (
            df_prev_ins_pos.groupby("SK_ID_CURR").agg(prev_agg_list).reset_index()
        )
        df_prev_ins_pos_agg.columns = pd.Index(
            [
                "PREV_" + col[0] + "_" + col[1].upper()
                for col in df_prev_ins_pos_agg.columns.tolist()
            ]
        )
        df_prev_ins_pos_agg.rename(
            columns={"PREV_SK_ID_CURR_": "SK_ID_CURR"}, inplace=True
        )
        # main table with prev_son
        df_prev_others = df.merge(df_prev_ins_pos_agg, how="left", on="SK_ID_CURR")

        # credit_card_balance
        df_prev_ins_pos_ccb = df_prev_others.merge(CCB_agg, how="left", on="SK_ID_CURR")

        # bureau_balance
        all_data = df_prev_ins_pos_ccb.merge(
            bureau_and_bb_agg, how="left", on="SK_ID_CURR"
        )

        print("all_data process:", all_data.shape)

    return all_data


# Create, Display, and Save Engineered Features

In [52]:
df = pre_processing_and_combine()

with pd.option_context("display.max_columns", None):
    display(df.head(10))

df.to_feather("engin_feats.ftr")


application train & test shape: (356251, 260)
Process application train - done in 2s
Bureau and Bureau Balance: (305811, 102)
Bureau and Bureau Balance - done in 10s
Installment Payments: (997752, 15)
Installment Payments - done in 15s
Pos Cash Balance: (936325, 15)
Pos Cash Balance - done in 12s
Credit Card Balance: (103558, 82)
Credit Card Balance - done in 21s
previous_application: (1670214, 114)
previous_application - done in 8s
all_data process: (356251, 777)
All tables are combining - done in 16s


,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,APP_REGION_POPULATION_RELATIVE,APP_DAYS_BIRTH,APP_DAYS_EMPLOYED,APP_DAYS_REGISTRATION,APP_DAYS_ID_PUBLISH,APP_OWN_CAR_AGE,APP_FLAG_MOBIL,APP_FLAG_EMP_PHONE,APP_FLAG_WORK_PHONE,APP_FLAG_CONT_MOBILE,APP_FLAG_PHONE,APP_FLAG_EMAIL,APP_CNT_FAM_MEMBERS,APP_REGION_RATING_CLIENT,APP_REGION_RATING_CLIENT_W_CITY,APP_HOUR_APPR_PROCESS_START,APP_REG_REGION_NOT_LIVE_REGION,APP_REG_REGION_NOT_WORK_REGION,APP_LIVE_REGION_NOT_WORK_REGION,APP_REG_CITY_NOT_LIVE_CITY,APP_REG_CITY_NOT_WORK_CITY,APP_LIVE_CITY_NOT_WORK_CITY,APP_EXT_SOURCE_1,APP_EXT_SOURCE_2,APP_EXT_SOURCE_3,APP_APARTMENTS_AVG,APP_BASEMENTAREA_AVG,APP_YEARS_BEGINEXPLUATATION_AVG,APP_YEARS_BUILD_AVG,APP_COMMONAREA_AVG,APP_ELEVATORS_AVG,APP_ENTRANCES_AVG,APP_FLOORSMAX_AVG,APP_FLOORSMIN_AVG,APP_LANDAREA_AVG,APP_LIVINGAPARTMENTS_AVG,APP_LIVINGAREA_AVG,APP_NONLIVINGAPARTMENTS_AVG,APP_NONLIVINGAREA_AVG,APP_APARTMENTS_MODE,APP_BASEMENTAREA_MODE,APP_YEARS_BEGINEXPLUATATION_MODE,APP_YEARS_BUILD_MODE,APP_COMMONAREA_MODE,APP_ELEVATORS_MODE,APP_ENTRANCES_MODE,APP_FLOORSMAX_MODE,APP_FLOORSMIN_MODE,APP_LANDAREA_MODE,APP_LIVINGAPARTMENTS_MODE,APP_LIVINGAREA_MODE,APP_NONLIVINGAPARTMENTS_MODE,APP_NONLIVINGAREA_MODE,APP_APARTMENTS_MEDI,APP_BASEMENTAREA_MEDI,APP_YEARS_BEGINEXPLUATATION_MEDI,APP_YEARS_BUILD_MEDI,APP_COMMONAREA_MEDI,APP_ELEVATORS_MEDI,APP_ENTRANCES_MEDI,APP_FLOORSMAX_MEDI,APP_FLOORSMIN_MEDI,APP_LANDAREA_MEDI,APP_LIVINGAPARTMENTS_MEDI,APP_LIVINGAREA_MEDI,APP_NONLIVINGAPARTMENTS_MEDI,APP_NONLIVINGAREA_MEDI,APP_TOTALAREA_MODE,APP_OBS_30_CNT_SOCIAL_CIRCLE,APP_DEF_30_CNT_SOCIAL_CIRCLE,APP_OBS_60_CNT_SOCIAL_CIRCLE,APP_DEF_60_CNT_SOCIAL_CIRCLE,APP_DAYS_LAST_PHONE_CHANGE,APP_FLAG_DOCUMENT_2,APP_FLAG_DOCUMENT_3,APP_FLAG_DOCUMENT_4,APP_FLAG_DOCUMENT_5,APP_FLAG_DOCUMENT_6,APP_FLAG_DOCUMENT_7,APP_FLAG_DOCUMENT_8,APP_FLAG_DOCUMENT_9,APP_FLAG_DOCUMENT_10,APP_FLAG_DOCUMENT_11,APP_FLAG_DOCUMENT_12,APP_FLAG_DOCUMENT_13,APP_FLAG_DOCUMENT_14,APP_FLAG_DOCUMENT_15,APP_FLAG_DOCUMENT_16,APP_FLAG_DOCUMENT_17,APP_FLAG_DOCUMENT_18,APP_FLAG_DOCUMENT_19,APP_FLAG_DOCUMENT_20,APP_FLAG_DOCUMENT_21,APP_AMT_REQ_CREDIT_BUREAU_HOUR,APP_AMT_REQ_CREDIT_BUREAU_DAY,APP_AMT_REQ_CREDIT_BUREAU_WEEK,APP_AMT_REQ_CREDIT_BUREAU_MON,APP_AMT_REQ_CREDIT_BUREAU_QRT,APP_AMT_REQ_CREDIT_BUREAU_YEAR,APP_NAME_CONTRACT_TYPE_Cash loans,APP_NAME_CONTRACT_TYPE_Revolving loans,APP_NAME_CONTRACT_TYPE_nan,APP_NAME_TYPE_SUITE_Children,APP_NAME_TYPE_SUITE_Family,APP_NAME_TYPE_SUITE_Group of people,APP_NAME_TYPE_SUITE_Other_A,APP_NAME_TYPE_SUITE_Other_B,"APP_NAME_TYPE_SUITE_Spouse, partner",APP_NAME_TYPE_SUITE_Unaccompanied,APP_NAME_TYPE_SUITE_nan,APP_NAME_INCOME_TYPE_Businessman,APP_NAME_INCOME_TYPE_Commercial associate,APP_NAME_INCOME_TYPE_Maternity leave,APP_NAME_INCOME_TYPE_Pensioner,APP_NAME_INCOME_TYPE_State servant,APP_NAME_INCOME_TYPE_Student,APP_NAME_INCOME_TYPE_Unemployed,APP_NAME_INCOME_TYPE_Working,APP_NAME_INCOME_TYPE_nan,APP_NAME_EDUCATION_TYPE_Academic degree,APP_NAME_EDUCATION_TYPE_Higher education,APP_NAME_EDUCATION_TYPE_Incomplete higher,APP_NAME_EDUCATION_TYPE_Lower secondary,APP_NAME_EDUCATION_TYPE_Secondary / secondary special,APP_NAME_EDUCATION_TYPE_nan,APP_NAME_FAMILY_STATUS_Civil marriage,APP_NAME_FAMILY_STATUS_Married,APP_NAME_FAMILY_STATUS_Separated,APP_NAME_FAMILY_STATUS_Single / not married,APP_NAME_FAMILY_STATUS_Unknown,APP_NAME_FAMILY_STATUS_Widow,APP_NAME_FAMILY_STATUS_nan,APP_NAME_HOUSING_TYPE_Co-op apartment,APP_NAME_HOUSING_TYPE_House / apartment,APP_NAME_HOUSING_TYPE_Municipal apartment,APP_NAME_HOUSING_TYPE_Office apartment,APP_NAME_HOUSING_TYPE_Rented apartment,APP_NAME_HOUSING_TYPE_With parents,APP_NAME_HOUSING_TYPE_nan,APP_OCCUPATION_TYPE_Accountants,APP_OCCUPATION_TYPE_Cleaning staff,APP_OCCUPATION_TYPE_Cooking staff,APP_OCCUPATION_TYPE_Core staff,APP_OCCUPATION_TYPE_Drivers,APP_OCCUPATION_TYPE_HR staff,APP_OCCUPATION_TYPE_High skill tech staff,APP_OCCU